# Limiting magnitude

The following series of images each show a grid of 7x7 stars with magnitudes ranging from 25 to 30 (Vega mags), observed in MCAO mode with the wide-field imaging mode of 4mas/pix, in Ks, H and J filters. The second image (with the colourful boxes) in each filter shows an estimate of signal-to-noise ratios (SNR) for each observed star. The final plots each show a fit through the magnitude and SNR values, with a limiting magnitude at SNR=5 plotted into the data.

In [ ]:
import numpy as np

import scopesim as sim
from scopesim import rc
from scopesim.source import source_templates as st

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from scipy.stats import linregress

import hmbp
from astropy import units as u
from itertools import product

# Set local path
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../"

args = [("open", "Ks", "open", 2),
        ("open", "H", "open", 2),
        ("J", "open", "open", 1),
        ("open", "Ks", "Br-gamma", 2)]
ao_modes = ("SCAO", "MCAO")
im_modes = ("IMG_1.5mas", "IMG_4mas")
dits = ((2.6, 17, 25), (1800, 22, 30), (18000, 24, 32))  # 2.6 s, 30 m, 5 h

In [ ]:
combos = list(product(ao_modes, im_modes, dits, args))
combos

In [ ]:
PLOTS = False
n_stars = 100

def lim_mag(ao_mode, im_mode, dit, arg):
    fw1, fw2, pw, r0 = arg
    dit, mmin, mmax = dit
    r1, r2 = 10, 15              # aperture radii  r0 (sig), r1-r2 (noise)
    w = 3 if im_mode == "IMG_4mas" else 1.125
    src = st.star_field(n_stars, mmin, mmax, width=w, use_grid=True)

    cmds = sim.UserCommands(use_instrument="MICADO",
                            set_modes=[ao_mode, im_mode])
    micado = sim.OpticalTrain(cmds)
    micado.cmds["!OBS.dit"] = dit
    micado["filter_wheel_1"].change_filter(fw1)
    micado["filter_wheel_2"].change_filter(fw2)
    micado["pupil_wheel"].change_filter(pw)
    micado["detector_linearity"].include = False

    micado.observe(src)
    hdul = micado.readout()[0]
    det = hdul[1].data
    imp = micado.image_planes[0].hdu.data  # e-/pixel/s

    if PLOTS:
        fig = plt.figure(figsize=(15, 20))
        ax1, ax2 = fig.subplots(2, 1)
        ax1.imshow(imp, norm=LogNorm(vmin=np.median(imp), vmax=1.01*np.median(imp)))
        ax2.imshow(det, norm=LogNorm(vmin=np.median(det), vmax=1.01*np.median(det)))

    offset = 2      # this needs to be addressed
    xpix, ypix = src.fields[0]["x"].data, src.fields[0]["y"].data
    scale = 0.004 if im_mode == "IMG_4mas" else 0.0015
    xpix = xpix / scale + 512 + offset
    ypix = ypix / scale + 512 + offset
    mags = np.round(np.linspace(mmin, mmax, n_stars), 1)

    snrs = []
    for x, y, mag in zip(xpix, ypix, mags):
        x, y = int(x+0.5), int(y+0.5)
        sig_im = np.copy(det[y-r0:y+r0+1, x-r0:x+r0+1])
        bg_im  = np.copy(det[y-r2:y+r2+1, x-r2:x+r2+1])
        bg_im[r1:-r1, r1:-r1] = 0

        bg_median = np.median(bg_im[bg_im > 0])
        bg_std = np.std(bg_im[bg_im > 0])
        # bg_std = np.sqrt(bg_median)
        noise = bg_std * np.sqrt(np.prod(sig_im.shape)) * np.sqrt(2)        # sqrt(2) comes from BG subtraction (see Rics doc)
        signal = np.sum(sig_im - bg_median)
        snr = signal/noise
        snrs += [snr]

        if PLOTS:
            plt.plot([x-r0, x+r0, x+r0, x-r0, x-r0],
                     [y-r0, y-r0, y+r0, y+r0, y-r0], "g")
            plt.plot([x-r1, x+r1, x+r1, x-r1, x-r1],
                     [y-r1, y-r1, y+r1, y+r1, y-r1], "y")
            plt.plot([x-r2, x+r2, x+r2, x-r2, x-r2],
                     [y-r2, y-r2, y+r2, y+r2, y-r2], "r")

            plt.text(x+r2, y+r2, str(round(snr, 1)), color="w")
            plt.text(x-r2, y-r2, str(mag), color="w")
    if fw1 != "open":
        fltr = fw1
    elif fw2 != "open":
        fltr = fw2
    else:
        fltr = pw
    if PLOTS:
        plt.title(f"Filter: {fltr}, mode: {im_mode}")
        plt.show()
    return mags, snrs, fltr

In [ ]:
def mag_fit(mags, snrs, fltr):
    snrs = np.array(snrs)
    results = linregress(mags[snrs > 5], np.log10(snrs[snrs > 5]))
    m, c = results[:2]
    sigma = 5
    lim_mag = (np.log10(sigma) - c) / m

    plt.plot(mags, snrs, ".", alpha=0.3)
    plt.plot(mags, 10 ** (m * mags + c), "r")

    plt.axhline(sigma)
    plt.axvline(lim_mag)
    fname = "".join(ff.title() for ff in fltr.split("-"))
    abmag = hmbp.convert(from_quantity=lim_mag*u.mag, to_unit=u.ABmag, filter_name=fname)
    txt = f"{lim_mag:.2f} Vega mag\n{abmag:.2f}"
    plt.text(lim_mag, sigma, txt,
             horizontalalignment="left",
             verticalalignment="bottom")
    plt.xlabel("Magnitude")
    plt.ylabel("SNR")
    plt.title(f"Filter: {fltr}")
    plt.show()
    return fltr, lim_mag, abmag.value

In [ ]:
lims = []
for combo in combos:
    mags, snrs, fltr = lim_mag(*combo)
    print(combo)
    print(min(mags), max(mags), min(snrs), max(snrs))
    lims.append(mag_fit(mags, snrs, fltr))

In [ ]:
def filtername(filters):
    fname = ""
    for fltr in filters:
        if fltr != "open" and isinstance(fltr, str):
            fname += fltr
    return fname

results = [(a, m, d[0], filtername(fltrs), *lim[1:]) for (a, m, d, fltrs), lim in zip(combos, lims)]

In [ ]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
idx = pd.MultiIndex.from_tuples([r[:4] for r in results],
      names=["AO", "Mode", "DIT", "Filter"]).reorder_levels(["AO", "Filter", "Mode", "DIT"])
df = pd.DataFrame([r[-2:] for r in results], columns=["Vega", "AB"], index=idx)
df.sort_values(by=["AO", "Filter", "Mode", "DIT"], ascending=[False, True, True, True])